In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.8 MB/s eta 0:00:00


In [4]:
# 필요한 함수, 모듈 업로드
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [29]:
df = pd.read_csv('/content/drive/MyDrive/bitamin_12_conf/독립변수/oil.csv')

df2 = pd.read_csv('/content/drive/MyDrive/bitamin_12_conf/탄소배출권/KAU19.csv')
# df

In [30]:
df

,Date,Dubai,Brent,WTI
0,2010-01-04,78.27,80.12,81.51
1,2010-01-05,79.83,80.59,81.77
2,2010-01-06,79.92,81.89,83.18
3,2010-01-07,80.76,81.51,82.66
4,2010-01-08,80.34,81.37,82.75
...,...,...,...,...
3756,2024-07-22,82.37,82.40,79.78
3757,2024-07-23,81.93,81.01,76.96
3758,2024-07-24,80.66,81.71,77.59
3759,2024-07-25,80.69,82.37,78.28


In [34]:
df2.rename(columns = {'일자' : 'Date'}, inplace = True)

In [35]:
for column in df.columns:
  if column != 'Date':
    df[column] = df[column] / df[column].shift()

# df2와 df를 'Date' 기준으로 합치기
df_combined = pd.merge(df, df2[['Date', '등락률']], on='Date', how='outer')

# NaN이 포함된 행 제거
df_combined.dropna(inplace=True)

df_combined.tail()

,Date,Dubai,Brent,WTI,등락률
2756,2020-09-07,0.938531,1.017312,0.961501,0.00
2757,2020-09-08,1.038094,0.961569,1.081882,1.19
2758,2020-09-09,0.988406,1.082871,1.035092,0.00
2759,2020-09-10,1.036656,0.957786,0.947055,-1.88
2760,2020-09-11,0.981957,1.012377,1.020928,-6.47


In [36]:
df_combined


,Date,Dubai,Brent,WTI,등락률
2580,2020-01-02,0.984351,1.040898,1.012139,-4.20
2581,2020-01-03,1.056790,1.031564,1.028544,-3.97
2582,2020-01-06,0.995610,0.970108,0.973727,-5.85
2583,2020-01-07,0.954989,0.986256,0.987545,6.67
2584,2020-01-08,1.032591,0.967533,0.959361,2.27
...,...,...,...,...,...
2756,2020-09-07,0.938531,1.017312,0.961501,0.00
2757,2020-09-08,1.038094,0.961569,1.081882,1.19
2758,2020-09-09,0.988406,1.082871,1.035092,0.00
2759,2020-09-10,1.036656,0.957786,0.947055,-1.88


## [auto_arima를 통한 ARIMA 모델 파라메터 설정]



In [37]:
# AutoARIMA 모델 학습(ARIMA) - 왜인지는 모르겠는데 변수로 지정하면 안 나오고, 이렇게 직접 입력하면 잘 나옴. 지금 AIC가 더 내려간 것으로 미루어 더 유의미하며, 심지어 decideCnt_diff_perc의 경우에도 alphba = 0.05 하에서 유의미
auto_arima_model = auto_arima(df_combined['등락률'], seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)

Arima_results = auto_arima_model.summary()

results_summary = str(Arima_results)
print(results_summary)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=843.026, Time=0.23 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=874.727, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=840.421, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=846.638, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=875.012, Time=0.02 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=842.336, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=842.363, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=844.421, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=839.555, Time=0.03 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=841.499, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=841.516, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=846.256, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=843.473, Time=0.13 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.972 seconds
                               SARI

## [auto_arima를 통한 ARIMAX 모델 파라메터 설정]

In [38]:
# AutoARIMA 모델 학습(ARIMAX) - 왜인지는 모르겠는데 변수로 지정하면 안 나오고, 이렇게 직접 입력하면 잘 나옴. 지금 AIC가 더 내려간 것으로 미루어 더 유의미하며, 심지어 decideCnt_diff_perc의 경우에도 alphba = 0.05 하에서 유의미
auto_arimax_model = auto_arima(df_combined['등락률'], df_combined[['Dubai']]
, seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)

Arimax_results = auto_arimax_model.summary()

results_summary = str(Arimax_results)
print(results_summary)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=845.020, Time=1.59 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=876.716, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=842.420, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=848.638, Time=0.25 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=874.718, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=844.335, Time=0.42 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=844.362, Time=0.34 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=846.354, Time=0.39 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=840.481, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=842.397, Time=0.09 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=842.423, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=846.674, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=844.481, Time=0.13 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 3.769 seconds
                               SARI

## [ARIMA 모델 학습 및 summary]

In [39]:
# ARIMA 모델 적합 및 결과 요약 출력을 위한 수정된 코드
from statsmodels.tsa.arima.model import ARIMA
Arima_Model = ARIMA(df_combined['등락률'], order = auto_arima_model.order,
                    enforce_stationarity=False,  # Depending on your data you might want to set this to False
                       enforce_invertibility=False)

# 모델 학습
Arima_results = Arima_Model.fit()
Arima_results_summary = Arima_results.summary()

# 결과를 문자열로 변환하여 라인별로 처리 가능하게 만듦
Arima_results_str = str(Arima_results_summary)
print(Arima_results_str)


                               SARIMAX Results                                
Dep. Variable:                    등락률   No. Observations:                  174
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -414.362
Date:                Wed, 31 Jul 2024   AIC                            834.725
Time:                        08:09:26   BIC                            844.185
Sample:                             0   HQIC                           838.563
                                - 174                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3276      0.402     -0.814      0.416      -1.116       0.461
ar.L1          0.4383      0.046      9.564      0.000       0.348       0.528
sigma2         7.0458      0.504     13.986      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


## [ARIMAX 모델 학습 및 summary]

In [40]:
# SARIMAX 모델 적합 및 결과 요약 출력을 위한 수정된 코드

# ARIMAX 모델 적합
Arimax_Model = SARIMAX(df_combined['등락률'],
                exog = df_combined['Dubai'],
                order = auto_arimax_model.order,
                       enforce_stationarity=False,  # Depending on your data you might want to set this to False
                       enforce_invertibility=False)

# 모델 학습
Arimax_results = Arimax_Model.fit()

Arimax_results_summary = Arimax_results.summary()

# 결과를 문자열로 변환하여 라인별로 처리 가능하게 만듦
Arimax_results_str = str(Arimax_results_summary)
print(Arimax_results_str)

                               SARIMAX Results                                
Dep. Variable:                    등락률   No. Observations:                  174
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -414.383
Date:                Wed, 31 Jul 2024   AIC                            834.765
Time:                        08:09:41   BIC                            844.225
Sample:                             0   HQIC                           838.603
                                - 174                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Dubai         -0.3120      0.398     -0.784      0.433      -1.092       0.468
ar.L1          0.4380      0.046      9.578      0.000       0.348       0.528
sigma2         7.0476      0.504     13.984      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
